##**Fine-tuning** **script**
This script:


*   Loads the Teacher-provided fine-tuning dataset
*   Processes and tokenizes
*   Loads Student models and tokenizer
*   Applies LoRA (PEFT)
*   Implements a training loop with supervised next-token prediction
*   Evaluates with validation loss
*   Saves LoRA adapter, tokenizer, and training logs

##**1. Imports**

In [ ]:
!pip install transformers peft accelerate bitsandbytes datasets pyyaml tqdm pandas

import os
import json
import random
import yaml
import pandas as pd
from dataclasses import dataclass, asdict
from typing import Dict, List, Optional

import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm import tqdm
from datetime import datetime

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    get_scheduler,
)

from peft import LoraConfig, get_peft_model

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 40.2 MB/s eta 0:00:00


##**2. Configuration**

In [ ]:
@dataclass
class FinetuneConfig:
    # Dataset paths (from Ene)
    train_file: str
    eval_file: str

    output_dir: str

    # Student model to be set later
    model_name: str

    dtype: str = "float16"
    device_map: str = "auto"
    max_length: int = 1024

    # LoRA settings
    # Note that only LoRA layers get updated
    lora_r: int = 8
    lora_alpha: int = 16
    lora_dropout: float = 0.05
    target_modules: Optional[List[str]] = None

    # Training
    epochs: int = 3
    batch_size: int = 4
    eval_batch_size: int = 8
    lr: float = 2e-4
    warmup_steps: int = 100
    weight_decay: float = 0.0
    grad_accum_steps: int = 4
    fp16: bool = True

    # We test loss every 100 steps
    eval_every_steps: int = 100

    # Checkpoint interval for longer runs
    save_every_steps: int = 500

    seed: int = 42

    def __post_init__(self):
        if self.target_modules is None:
            self.target_modules = ["q_proj", "v_proj"]

##**3. Dataset Loading**

In [ ]:
# We use Teacher-generated (Q, R) pairs as training and evaluation data.

def load_jsonl(path: str):
    data = []
    with open(path, "r") as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    return data


def load_dataset(train_path: str, eval_path: str):
    # Normalize format to {question, response}
    train_raw = load_jsonl(train_path)
    eval_raw = load_jsonl(eval_path)

    train = [{"prompt": x["prompt"], "response": x["response"]} for x in train_raw]
    eval = [{"prompt": x["prompt"], "response": x["response"]} for x in eval_raw]

    return train, eval

##**4. Tokenization**

In [ ]:
# During supervised fine-tuning, we compute cross-entropy loss of response given the prompt.
# We mask prompt tokens with -100 so the loss ignores the prompt and applies only to response tokens.

def tokenize_pair(tokenizer, question, response, max_length):
    eos = tokenizer.eos_token
    q_with_eos = question + eos
    full_text = q_with_eos + response + eos

    # Tokenize separately so we know the boundary between prompt and response
    enc_q = tokenizer(q_with_eos, add_special_tokens=False)
    enc_full = tokenizer(full_text, truncation=True, max_length=max_length, add_special_tokens=False)

    input_ids = enc_full.input_ids
    q_len = len(enc_q.input_ids)

    # Masking such that only response tokens contribute to cross-entropy
    labels = [-100] * q_len + input_ids[q_len:]
    labels = labels[:len(input_ids)]

    return {
        "input_ids": input_ids,
        "attention_mask": enc_full.attention_mask,
        "labels": labels,
    }

class QRPairsDataset(Dataset):
    """
    Dataset for Q -> R supervised fine-tuning.
    """

    def __init__(self, records, tokenizer, max_length):
        self.records = records
        self.tok = tokenizer
        self.max_len = max_length

    def __len__(self):
        return len(self.records)

    def __getitem__(self, idx):
        r = self.records[idx]
        return tokenize_pair(self.tok, r["prompt"], r["response"], self.max_len)

##**5. Batch Collation**

In [ ]:
def collate_fn(batch, pad_token_id):
    max_len = max(len(x["input_ids"]) for x in batch)

    padded_inputs, padded_masks, padded_labels = [], [], []

    for item in batch:
        pad = max_len - len(item["input_ids"])

        padded_inputs.append(item["input_ids"] + [pad_token_id] * pad)
        padded_masks.append(item["attention_mask"] + [0] * pad)
        padded_labels.append(item["labels"] + [-100] * pad)  # we keep masked tokens masked

    return {
        "input_ids": torch.tensor(padded_inputs),
        "attention_mask": torch.tensor(padded_masks),
        "labels": torch.tensor(padded_labels),
    }

##**6. Load Student Model and LoRA**

In [ ]:
# We perform supervised LoRA fine-tuning using HuggingFace PEFT.
# Only LoRA adapter weights are updated. The entire base model stays frozen.

def load_student_model(cfg: FinetuneConfig):
    dtype_map = {
        "float16": torch.float16,
        "bfloat16": torch.bfloat16,
        "float32": torch.float32,
    }

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(cfg.model_name, use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Load Student model
    model = AutoModelForCausalLM.from_pretrained(
        cfg.model_name,
        torch_dtype=dtype_map[cfg.dtype],
        device_map=cfg.device_map,
    )

    # LoRA
    lora_cfg = LoraConfig(
        r=cfg.lora_r,
        lora_alpha=cfg.lora_alpha,
        lora_dropout=cfg.lora_dropout,
        target_modules=cfg.target_modules,
        task_type="CAUSAL_LM",
    )

    model = get_peft_model(model, lora_cfg)
    return tokenizer, model

##**7. Evaluation**

In [ ]:
# For evaluation, we compute cross-entropy loss over response tokens (every 100 steps)

def evaluate(model, dataloader, device):
    model.eval()
    total, count = 0.0, 0

    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            out = model(**batch)
            total += out.loss.item()
            count += 1

    model.train()
    return total / max(1, count)

##**8. Fine-Tuning Loop**

In [ ]:
# This cell implements the following procedure:
# 1. Compute cross-entropy loss of responses given prompts
# 2. Backpropagate to update LoRA adapter weights
# 3. Record training loss every step
# 4. Compute testing loss every 100 steps
# 5. Testing loss is used as the internalization metric
# 6. Logging supports later plotting of training/testing curves

def finetune(cfg: FinetuneConfig):

    random.seed(cfg.seed)
    torch.manual_seed(cfg.seed)

    os.makedirs(cfg.output_dir, exist_ok=True)

    # Load datasets
    train_records, eval_records = load_dataset(cfg.train_file, cfg.eval_file)

    # Load Student Model with LoRA adapters
    tokenizer, model = load_student_model(cfg)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    pad_id = tokenizer.pad_token_id

    # Dataset and DataLoader
    train_ds = QRPairsDataset(train_records, tokenizer, cfg.max_length)
    eval_ds = QRPairsDataset(eval_records, tokenizer, cfg.max_length)

    train_loader = DataLoader(
        train_ds, batch_size=cfg.batch_size, shuffle=True,
        collate_fn=lambda b: collate_fn(b, pad_id)
    )

    eval_loader = DataLoader(
        eval_ds, batch_size=cfg.eval_batch_size, shuffle=False,
        collate_fn=lambda b: collate_fn(b, pad_id)
    )

    # Optimizer (on LoRA parameters only)
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = AdamW(params, lr=cfg.lr, weight_decay=cfg.weight_decay)

    total_steps = (len(train_loader) * cfg.epochs) // cfg.grad_accum_steps
    scheduler = get_scheduler(
        "linear", optimizer=optimizer,
        num_warmup_steps=cfg.warmup_steps,
        num_training_steps=total_steps,
    )

    scaler = torch.cuda.amp.GradScaler(enabled=cfg.fp16)

    logs = []
    global_step = 0
    model.train()

    for ep in range(cfg.epochs):
        print(f"Starting epoch {ep+1}/{cfg.epochs}")

        for step, batch in enumerate(tqdm(train_loader)):
            batch = {k: v.to(device) for k, v in batch.items()}

            # Compute cross-entropy loss
            with torch.cuda.amp.autocast(enabled=cfg.fp16):
                out = model(**batch)
                loss = out.loss / cfg.grad_accum_steps

            # Backprop into LoRA weights only
            scaler.scale(loss).backward()

            # Update after gradient accumulation
            if (step + 1) % cfg.grad_accum_steps == 0:
                scaler.unscale_(optimizer)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                scheduler.step()
                global_step += 1

                # Log training loss
                logs.append({"step": global_step, "train_loss": out.loss.item()})

                # Compute testing loss every 100 steps
                if global_step % cfg.eval_every_steps == 0:
                    val_loss = evaluate(model, eval_loader, device)
                    logs.append({"step": global_step, "eval_loss": val_loss})
                    print(f"Step {global_step}: val_loss = {val_loss:.4f}")

                # Checkpointing
                if global_step % cfg.save_every_steps == 0:
                    ckpt_dir = os.path.join(cfg.output_dir, f"checkpoint-{global_step}")
                    os.makedirs(ckpt_dir, exist_ok=True)
                    model.save_pretrained(ckpt_dir)
                    tokenizer.save_pretrained(ckpt_dir)

    # Save final Student Model and loss logs
    model.save_pretrained(cfg.output_dir)
    tokenizer.save_pretrained(cfg.output_dir)
    pd.DataFrame(logs).to_csv(os.path.join(cfg.output_dir, "training_logs.csv"), index=False)
    print("Fine-tuning complete!")

##**8.5 Dummy Testing Data** (we'll swap for actual datasets)

In [ ]:
# dummy teacher datasets

base = "/content/datasets_dummy"
os.makedirs(base, exist_ok=True)

teacher_template_train = [
    {"prompt": "Explain gravity.",
     "response": "Gravity is the force that attracts objects toward each other."},
    {"prompt": "Define photosynthesis.",
     "response": "Photosynthesis is the process plants use to convert sunlight into energy."}
]

teacher_template_eval = [
    {"prompt": "What is an atom?",
     "response": "An atom is the smallest unit of matter."}
]

teacher_baseline_train = [
    {"prompt": "Write a sentence about the ocean.",
     "response": "The ocean is vast and full of mysteries."},
    {"prompt": "Describe a cat.",
     "response": "A cat is a furry domestic animal with whiskers and claws."}
]

teacher_baseline_eval = [
    {"prompt": "What is a tree?",
     "response": "A tree is a tall plant with a trunk and branches."}
]

files = {
    "teacher1_template_train.jsonl": teacher_template_train,
    "teacher1_template_eval.jsonl": teacher_template_eval,
    "teacher1_baseline_train.jsonl": teacher_baseline_train,
    "teacher1_baseline_eval.jsonl": teacher_baseline_eval,
}

for filename, rows in files.items():
    path = os.path.join(base, filename)
    with open(path, "w") as f:
        for row in rows:
            f.write(json.dumps(row) + "\n")

print("Dummy datasets created in:", base)
print("Files:", os.listdir(base))

Dummy datasets created in: /content/datasets_dummy
Files: ['teacher1_template_train.jsonl', 'teacher1_baseline_eval.jsonl', 'teacher1_template_eval.jsonl', 'teacher1_baseline_train.jsonl']


##**9. Fine-Tuning Runs**

In [ ]:
TEACHER_DATASETS = [
    {
        "name": "teacher1_template",
        "train": "/content/datasets_dummy/teacher1_template_train.jsonl",
        "eval":  "/content/datasets_dummy/teacher1_template_eval.jsonl",
    },
    {
        "name": "teacher1_baseline",
        "train": "/content/datasets_dummy/teacher1_baseline_train.jsonl",
        "eval":  "/content/datasets_dummy/teacher1_baseline_eval.jsonl",
    },      # repeat for other teacher datasets
]

STUDENT_MODELS = [
    "Qwen/Qwen2.5-7B-Instruct",
    #"meta-llama/Llama-2-7b-chat-hf", WE NEED ACCESS HERE
]

for student in STUDENT_MODELS:
    student_name = student.split("/")[-1]

    for teacher_dataset in TEACHER_DATASETS:
        teacher_name = teacher_dataset["name"]

        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
        output_dir = f"/content/runs/{student_name}_{teacher_name}_{timestamp}"

        cfg = FinetuneConfig(
            train_file=teacher_dataset["train"],
            eval_file=teacher_dataset["eval"],
            model_name=student,
            output_dir=output_dir,

            batch_size=4,
            eval_batch_size=8,
            max_length=1024,
            eval_every_steps=100,
        )

        print("\n=====================================")
        print(f"Starting run: Student={student_name}, Teacher={teacher_name}")
        print(f"Saving to: {output_dir}")
        print("=====================================\n")

        finetune(cfg)

print("=== ALL RUNS COMPLETE ===")


Starting run: Student=Qwen2.5-7B-Instruct, Teacher=teacher1_template
Saving to: /content/runs/Qwen2.5-7B-Instruct_teacher1_template_20251119-004734



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/tmp/ipython-input-1926694637.py:50: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=cfg.fp16)


Starting epoch 1/3


  0%|          | 0/1 [00:00<?, ?it/s]/tmp/ipython-input-1926694637.py:63: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=cfg.fp16):
100%|██████████| 1/1 [00:00<00:00,  6.70it/s]


...

Starting epoch 2/3


100%|██████████| 1/1 [00:00<00:00,  6.28it/s]


Starting epoch 3/3


100%|██████████| 1/1 [00:00<00:00,  6.70it/s]


Fine-tuning complete!
=== ALL RUNS COMPLETE ===
